In [26]:
import asyncio
import random
from agents import Agent, ItemHelpers, Runner, function_tool
from dotenv import load_dotenv
import os
from openai.types.responses import ResponseTextDeltaEvent
load_dotenv()


True

In [27]:
@function_tool
def how_many_jokes() -> int:
    return 1

In [28]:
from pydantic import BaseModel, Field

class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline of the joke")


In [45]:
async def test():
    agent = Agent(
        name="Joker",
        instructions="First call the `how_many_jokes` tool, then tell that many jokes.",
        tools=[how_many_jokes],
        output_type=Joke,
    )

    result = Runner.run_streamed(
        agent,
        input="Hello",
    )
    print("=== Run starting ===")

    async for event in result.stream_events():
        # We'll ignore the raw responses event deltas
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            # print("Event delta")
            print(event.data.delta, end="", flush=True)
        # When the agent updates, print that
        elif event.type == "agent_updated_stream_event":
            print(f"Agent updated: {event.new_agent.name}")
            continue
        # When items are generated, print them
        elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                print(f"-- Tool called: {event.item.raw_item.name}")
                print(f"-- Tool args: {event.item.raw_item.arguments}")
                print("-- Tool was called")
            elif event.item.type == "tool_call_output_item":
                print(f"-- Tool output: {event.item.output}")
            elif event.item.type == "message_output_item":
                print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
                output = ItemHelpers.text_message_output(event.item)
                parse_output = Joke.model_validate_json(output)
                print(f"Joke setup: {parse_output.setup}")
                print(f"Joke punchline: {parse_output.punchline}")
            else:
                pass  # Ignore other event types
        else:
            print(f"Unknown event type: {event.type}")

    print("=== Run complete ===")

In [46]:
output = await test()
print(output)

=== Run starting ===
Agent updated: Joker
Unknown event type: raw_response_event
Unknown event type: raw_response_event
Unknown event type: raw_response_event
Unknown event type: raw_response_event
Unknown event type: raw_response_event
Unknown event type: raw_response_event
Unknown event type: raw_response_event
-- Tool called: how_many_jokes
-- Tool args: {}
-- Tool was called
-- Tool output: 1
Unknown event type: raw_response_event
Unknown event type: raw_response_event
Unknown event type: raw_response_event
Unknown event type: raw_response_event
{"setup":"Why don't skeletons fight each other?","punchline":"They don't have the guts."}Unknown event type: raw_response_event
Unknown event type: raw_response_event
Unknown event type: raw_response_event
Unknown event type: raw_response_event
-- Message output:
 {"setup":"Why don't skeletons fight each other?","punchline":"They don't have the guts."}
Joke setup: Why don't skeletons fight each other?
Joke punchline: They don't have the gut